#### Mahri


In [ ]:
# # On g collab only
# !git clone https://github.com/electronjia/heart_cardiac_mri_image_processing.git

In [ ]:
# # On g collab only
# !pip install pydicom

In [ ]:
# # On g collab only
# !pwd
# !ls -l
# %cd heart_cardiac_mri_image_processing/edge_detection_and_contours
# !pwd

In [ ]:
import pandas as pd
from config import *
import os
import numpy as np
from skimage import exposure, filters, measure, segmentation
import matplotlib.pyplot as plt
import pydicom
from skimage import img_as_ubyte
from skimage.feature import canny
from skimage.measure import label, regionprops
from IPython.display import display
from skimage.draw import polygon
from skimage.morphology import convex_hull_image


In [ ]:
# # On g collab only
# import sys
# sys.path.append('/content/heart_cardiac_mri_image_processing/edge_detection_and_contours')
# user_handle = r"/content/heart_cardiac_mri_image_processing/data"
# patient_data_excel_path = r"/content/heart_cardiac_mri_image_processing/scd_patientdata_xlsx.xlsx"
# patient_data = "patient_data"
# patient_filepaths = "patient_filepaths"

# user_handle = "/content/heart_cardiac_mri_image_processing/data"

In [ ]:
patient_xlsx = patient_data_excel_path
patient_data_df = pd.read_excel(patient_xlsx, sheet_name=patient_data)
patient_filepaths_df = pd.read_excel(patient_xlsx, sheet_name=patient_filepaths)
patient_mask_filepaths = pd.read_excel(patient_xlsx, sheet_name="mask_filepaths")

display(patient_data_df.head(2))
display(patient_filepaths_df.head(2))

In [ ]:
def read_convert_dicom_img(filepath):
    # Read the DICOM image
    dicom_img = pydicom.dcmread(filepath)

    # Extract the pixel data and convert to 8-bit image
    img = img_as_ubyte(dicom_img.pixel_array / np.max(dicom_img.pixel_array))
    return img

In [ ]:
def apply_contrast_enhancement(img, clip_limit):
    # Apply contrast
    img_contrast = exposure.equalize_adapthist(img, clip_limit=clip_limit)

    # Create radial mask for center contrast enhancement
    # Create a grid of distances from the center
    rows, cols = img_contrast.shape
    center_row, center_col = rows//2, cols//2

    # Create radial distance map
    y, x = np.ogrid[:rows, :cols]
    distance_from_center = np.sqrt((x - center_col)**2 + (y - center_row)**2)

    # Normalize distance to range [0, 1]
    distance_from_center = distance_from_center / np.max(distance_from_center)

    # Apply the mask (higher values in the center)
    radial_mask = 1 - distance_from_center  # Invert to get higher values at the center
    enhanced_contrast_img = img_contrast * radial_mask

    return enhanced_contrast_img

In [ ]:
def detect_edges_and_contours(img, sigma, level):
    # Edge detection
    img_edges = canny(img, sigma=sigma)

    # Find and draw contours
    contours = measure.find_contours(img_edges, level=level)

    # Create a binary mask with contours
    binary_mask = np.zeros_like(img, dtype=np.uint8)
    for contour in contours:
        # Convert contour to polygon coordinates and fill it in the binary image
        contour_points = contour.astype(int)
        rr, cc = polygon(contour_points[:, 0], contour_points[:, 1], shape=binary_mask.shape)
        binary_mask[rr, cc] = 255  # Set contour area to foreground (255)

    return img_edges, contours, binary_mask

In [ ]:
def get_labels_regions(binary_mask):
    # Find the labels in binary mask
    labeled_img, _ = measure.label(binary_mask, connectivity=2, return_num=True)

    # Measure region properties
    regions = measure.regionprops(labeled_img)

    return labeled_img, regions

In [ ]:
def segment_left_ventricle(filepath, labeled_img, regions, mask_info_dict, img_idx, seed_starter, eccentricity_th, area_th, distance_th):

  # Find best region depending on previous best region or
  min_distance = float('inf')
  best_region = None

  # Attempt to get the best region by comparing to previous region if exists, if not, use seed starter
  for prop in regions:

      # Use the set seed starter
      previous_centroid = np.array(seed_starter)
      current_centroid = np.array(prop.centroid)
      distance = np.linalg.norm(current_centroid - previous_centroid)  # Euclidean distance

      if distance < min_distance:
        min_distance = distance
        best_region = prop


  # Attempt to evaluate the best region's eccentricity and area according to set thresholds
  try:
    if best_region.eccentricity < eccentricity_th and best_region.area < area_th and min_distance < distance_th:
      filled_mask = get_convex_hull_mask(labeled_img, best_region)
    else:
      filled_mask = np.zeros_like(labeled_img, dtype=np.uint8)

    # Append the dictionary info
    mask_info_dict["index"].append(img_idx)
    mask_info_dict["frames"].append(filepath)
    mask_info_dict["eccentricity"].append(best_region.eccentricity)
    mask_info_dict["area"].append(best_region.area)
    mask_info_dict["coords"].append(best_region.coords)
    mask_info_dict["centroid_coords"].append(best_region.centroid)
    mask_info_dict["distance_centroid"].append(min_distance)
    mask_info_dict['mask'].append(filled_mask)

  except:
    filled_mask = np.zeros_like(labeled_img, dtype=np.uint8)
    # Append the dictionary info
    mask_info_dict["index"].append(img_idx)
    mask_info_dict["frames"].append(filepath)
    mask_info_dict["eccentricity"].append(0)
    mask_info_dict["area"].append(0)
    mask_info_dict["coords"].append(0)
    mask_info_dict["centroid_coords"].append((0,0))
    mask_info_dict["distance_centroid"].append(0)
    mask_info_dict['mask'].append(filled_mask)


  return filled_mask, mask_info_dict

In [ ]:
def segment_left_ventricle_snakes(filepath, labeled_img, regions, mask_info_dict, img_idx, seed_starter, eccentricity_th, distance_th):

  # Find best region depending on previous best region or
  min_distance = float('inf')
  best_region = None

  # Attempt to get the best region by comparing to previous region if exists, if not, use seed starter
  for prop in regions:
      # print(f"Region eccentricity: {prop.eccentricity}, area: {prop.area}")

      # Use the set seed starter
      previous_centroid = np.array(seed_starter)
      current_centroid = np.array(prop.centroid)
      distance = np.linalg.norm(current_centroid - previous_centroid)  # Euclidean distance

      if distance < min_distance:
        min_distance = distance
        best_region = prop

  # Attempt to evaluate the best region's eccentricity and area according to set thresholds
  try:
    if best_region.eccentricity < eccentricity_th and min_distance < distance_th:
      # print(f"New region eccentricity {best_region.eccentricity:.3f} and area {best_region.area:.3f} and distance {min_distance}")
      filled_mask = get_filled_mask(labeled_img, best_region)
    else:
      filled_mask = np.zeros_like(labeled_img, dtype=np.uint8)

    # Append the dictionary info
    mask_info_dict["index"].append(img_idx)
    mask_info_dict["frames"].append(filepath)
    mask_info_dict["eccentricity"].append(best_region.eccentricity)
    mask_info_dict["area"].append(best_region.area)
    mask_info_dict["coords"].append(best_region.coords)
    mask_info_dict["centroid_coords"].append(best_region.centroid)
    mask_info_dict["distance_centroid"].append(min_distance)
    mask_info_dict['mask'].append(filled_mask)

  except:
    filled_mask = np.zeros_like(labeled_img, dtype=np.uint8)
    # Append the dictionary info
    mask_info_dict["index"].append(img_idx)
    mask_info_dict["frames"].append(filepath)
    mask_info_dict["eccentricity"].append(0)
    mask_info_dict["area"].append(0)
    mask_info_dict["coords"].append(0)
    mask_info_dict["centroid_coords"].append((0,0))
    mask_info_dict["distance_centroid"].append(0)
    mask_info_dict['mask'].append(filled_mask)


  return filled_mask, mask_info_dict

In [ ]:
def get_convex_hull_mask(labeled_img, region):

    # Create a mask for smallest eccentricity region
    binary_mask = np.zeros_like(labeled_img, dtype=np.uint8)
    binary_mask[labeled_img == region.label] = 255

    # Generate convex hull mask
    hull_mask = convex_hull_image(binary_mask)

    # Find contours in the binary mask
    contours = measure.find_contours(hull_mask, level=0.5)

    # Obtain the filled mask
    filled_mask = np.zeros_like(binary_mask, dtype=np.uint8)

    # Loop through each contour and fill it in the mask
    for contour in contours:
        # Convert contour coordinates to integer values
        contour_points = contour.astype(int)

        # Get the coordinates of the contour and fill the polygon
        rr, cc = polygon(contour_points[:, 0], contour_points[:, 1], shape=filled_mask.shape)

        # Set the region inside the polygon to 255 (foreground)
        filled_mask[rr, cc] = 255  # Set filled region to white (255)

    return filled_mask

In [ ]:
def get_filled_mask(labeled_img, region):

    # Create a mask for smallest eccentricity region
    binary_mask = np.zeros_like(labeled_img, dtype=np.uint8)
    binary_mask[labeled_img == region.label] = 255

    # Find contours in the binary mask
    contours = measure.find_contours(binary_mask, level=0.5)

    # Obtain the filled mask
    filled_mask = np.zeros_like(binary_mask, dtype=np.uint8)

    # Loop through each contour and fill it in the mask
    for contour in contours:
        # Convert contour coordinates to integer values
        contour_points = contour.astype(int)

        # Get the coordinates of the contour and fill the polygon
        rr, cc = polygon(contour_points[:, 0], contour_points[:, 1], shape=filled_mask.shape)

        # Set the region inside the polygon to 255 (foreground)
        filled_mask[rr, cc] = 255  # Set filled region to white (255)

    return filled_mask

In [ ]:
def store_evolution_in(lst):
    """Returns a callback function to store the evolution of the level sets in
    the given list.
    """

    def _store(x):
        lst.append(np.copy(x))

    return _store

In [ ]:
# patient_thresholds = {
#     'eccentricity': [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8,0.8, 0.9, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8],
#     'area': [300, 300, 400, 300, 400, 400, 400, 500, 500, 300, 300, 400, 400, 300, 300, 400, 400, 400, 400, 300, 300, 300, 300, 400, 300, 400, 400, 400, 400, 300, 300, 400, 400, 300, 400, 400, 400, 300, 400, 300, 400, 300, 400, 300, 300],
#     'seed_starter' : [(130, 130), (130,130), (130, 130), (120, 120), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (120, 120), (130, 130), (130, 130), (130, 130), (130, 150), (130, 150), (130, 110), (130, 130), (110, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (130, 130), (120, 130), (130, 130), (130, 150), (130, 130)],
#     # Decrese sigma gaus to make it less smooth and more pronounced
#     'sigma_gaus': [0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05],
#     # Increase clip limit to increase contrast
#     'clip_limit': [0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.07,0.03,0.07,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03],
#     # Increase sigma edge to get rid of some unwanted edges
#     'sigma_edge': [6.30,6.3,6.30,5.00,6.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,6.00,6.00,6.00,6.00,6.00,6.0,6.0,6.50,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.50,6.00,5.50,5.50],
#     'level': [0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7,0.7],
#     'distance_centroid_th': [20,20,20,20,20,20,20,20,20,30,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,10,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20],
# }

patient_ids = patient_filepaths_df['patient_id'].unique()

patients_idxs = range(0, len(patient_ids))

print(f"Number of patients: {len(patient_ids)}")

In [ ]:
for patient_idx, patient_id in zip(patients_idxs,patient_ids):
  print(patient_idx, patient_id)

In [ ]:
from IPython.display import display, Markdown
import json
# Number of columns for plotting
num_cols = 10


patient_mask_info = []

with open("patient_thresholds.txt", "r") as file:
    patient_thresholds = json.load(file)


for patient_idx, patient_id in zip(patients_idxs,patient_ids):
  single_patient_filepaths = patient_filepaths_df.loc[patient_filepaths_df['patient_id'] == patient_id, 'dcm_image_filepath'].tolist()


  # print(f"Processing patient: {patient_id} with index of {patient_idx}")

  display(Markdown(f"# Processing patient: **{patient_id}** with index of **{patient_idx}**"))


  img_processing_params = patient_thresholds[f"patient_{patient_idx}"]
  print(img_processing_params)

  mask_info_dict = {
      "index": [],
      "frames": [],
      "eccentricity": [],
      "area": [],
      "coords": [],
      "centroid_coords": [],
      "distance_centroid": [],
      "mask": []
  }
  
  for batch_idx, batch_start in enumerate(range(0, len(single_patient_filepaths), num_cols)):
    
    batch_filepaths = single_patient_filepaths[batch_start:batch_start + num_cols]

    num_imgs = len(batch_filepaths)
    fig, axes = plt.subplots(1, num_imgs, figsize=(num_imgs * 2, 5))

    if num_imgs == 1:
        axes = [axes]  # Ensure axes is iterable when there's only one image

    # Iterate over the filepaths in given batch
    for batch_img_idx, img_filepath in enumerate(batch_filepaths):

        # Get the original image index
        img_idx = int(f"{batch_idx}{batch_img_idx}")

        # For Google Colab only
        #   img_filepath = img_filepath.replace("\\", "/")
        img_abs_filepath = os.path.join(user_handle, img_filepath)

        # Get the 8 bit image
        img_8bit = read_convert_dicom_img(img_abs_filepath)

        # Get filtered image
        img_filt = filters.gaussian(img_8bit, sigma=img_processing_params['sigma_gaus'])

        # Apply radial contrast on image
        img_contrast = apply_contrast_enhancement(img_filt, img_processing_params['clip_limit'])

        # Snakes
        try:
            if img_processing_params['snakes_radius']:
               snakes_radius_ex = img_processing_params['snakes_radius']
        
            # Initial level set
            init_ls = segmentation.disk_level_set(img_contrast.shape, center=img_processing_params['seed_starter'], radius=img_processing_params['snakes_radius'])

            # Store evolution of segmentation
            evolution = []
            callback = store_evolution_in(evolution)
            ls = segmentation.morphological_chan_vese(img_contrast, num_iter=img_processing_params['snakes_iter'], init_level_set=init_ls, iter_callback=callback)

            binary_mask = evolution[img_processing_params['snakes_iter']]  # ACWE result

            # Get labeled image and regions
            labeled_img, regions = get_labels_regions(binary_mask)

            # Get the filled mask where left ventricle is defined
            filled_mask, mask_info_dict = segment_left_ventricle_snakes(filepath=img_abs_filepath, labeled_img=labeled_img, regions=regions, mask_info_dict=mask_info_dict, img_idx=img_idx, seed_starter=img_processing_params['seed_starter'], eccentricity_th = img_processing_params['eccentricity'],distance_th=img_processing_params['distance_centroid_th'])

        except:

            # Get edges, contours, and binary mask
            img_edges, contours, binary_mask = detect_edges_and_contours(img_contrast, sigma=img_processing_params['sigma_edge'], level=img_processing_params['level'])

            # Get labeled image and regions
            labeled_img, regions = get_labels_regions(binary_mask)
            # print(regions)

            # Get the filled mask where left ventricle is defined
            filled_mask, mask_info_dict = segment_left_ventricle(filepath=img_abs_filepath, labeled_img=labeled_img, regions=regions, mask_info_dict=mask_info_dict, img_idx=img_idx, seed_starter=img_processing_params['seed_starter'], eccentricity_th = img_processing_params['eccentricity'], area_th=img_processing_params['area'], distance_th=img_processing_params['distance_centroid_th'])

        # Plot the overlay
        # Normalize original image for proper display (optional)
        img_plot = (img_8bit - img_8bit.min()) / (img_8bit.max() - img_8bit.min())  # Normalize to [0,1] range

        # Create an RGB version of the grayscale image
        img_rgb = np.stack([img_plot] * 3, axis=-1)  # Convert grayscale to RGB (shape: HxWx3)

        # Create a red-colored mask (overlay will be red where the mask is)
        mask_rgb = np.zeros_like(img_rgb)  # Create an empty RGB image

        mask_rgb[:, :, 0] = filled_mask  # Red channel

        # Alpha blending
        alpha = 0.05  # Transparency level
        overlay = (img_rgb * (1 - alpha) + mask_rgb * alpha)

        # Ensure values are in valid range [0,1] for display
        overlay = np.clip(overlay, 0, 1)

        # Plot processed image
        centroid_plot = f"C:{mask_info_dict['centroid_coords'][img_idx][0]:.0f},{mask_info_dict['centroid_coords'][img_idx][1]:.0f}"
        # axes[batch_img_idx].imshow(img_contrast, cmap="gray", vmin=np.min(img_contrast), vmax=np.max(img_contrast))
        # axes[batch_img_idx].imshow(binary_mask, cmap="gray", vmin=np.min(binary_mask), vmax=np.max(binary_mask), alpha=0.5)
        # axes[batch_img_idx].set(title=f" Fr {img_idx}: E:{mask_info_dict['eccentricity'][img_idx]:.1f}, \nA:{mask_info_dict['area'][img_idx]:.0f}, C:{centroid_plot}")

        # Plot final segmentation result
        axes[batch_img_idx].imshow(overlay, cmap="gray", vmin=np.min(overlay), vmax=np.max(overlay))
        axes[batch_img_idx].imshow(binary_mask, cmap="gray", vmin=np.min(binary_mask), vmax=np.max(binary_mask), alpha=0.2)
        axes[batch_img_idx].set(title=f" Fr {img_idx}: E:{mask_info_dict['eccentricity'][img_idx]:.1f}, \nA:{mask_info_dict['area'][img_idx]:.0f}, DC: {mask_info_dict['distance_centroid'][img_idx]:.1f}, \n{centroid_plot}")

    plt.tight_layout()
    plt.show()
patient_mask_info.append(mask_info_dict)


In [ ]:
# Saving patient thresholds text file

patient_thresholds = {
    f"patient_{i}": {
        "eccentricity": ecc,
        "area": area,
        "seed_starter": seed,
        "sigma_gaus": sigma_gaus,
        "clip_limit": clip_limit,
        "sigma_edge": sigma_edge,
        "level": level,
        "distance_centroid_th": distance_centroid
    }
    for i, (ecc, area, seed, sigma_gaus, clip_limit, sigma_edge, level, distance_centroid) in enumerate(zip(
        patient_thresholds["eccentricity"],
        patient_thresholds["area"],
        patient_thresholds["seed_starter"],
        patient_thresholds["sigma_gaus"],
        patient_thresholds["clip_limit"],
        patient_thresholds["sigma_edge"],
        patient_thresholds["level"],
        patient_thresholds["distance_centroid_th"]
    ))
}

import json

# Save dictionary to a text file
with open("patient_thresholds.txt", "w") as file:
    json.dump(patient_thresholds, file, indent=4)

# Load dictionary from the text file
with open("patient_thresholds.txt", "r") as file:
    loaded_dict = json.load(file)

